## Australian Rainfall Data Download and Exploratory Data Analysis

Attribution: code adapted from DSCI 525 lecture notes.

#### 1. Imports 

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np

#### 2. Downloading data using figshare API

In [2]:
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text) 
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 8.36 s, sys: 14.6 s, total: 23 s
Wall time: 8min 18s


In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 18.2 s, sys: 2.96 s, total: 21.2 s
Wall time: 23 s


#### 3. Combining data files into one CSV file

In [8]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r"[\/|\\](.*)_daily_rainfall", file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 5min 51s, sys: 25.2 s, total: 6min 16s
Wall time: 6min 34s


In [9]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.6G	figsharerainfall/combined_data.csv


In [10]:
%%time
df.head()

CPU times: user 265 µs, sys: 732 µs, total: 997 µs
Wall time: 1.26 ms


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


##### 3.1 Comparing run times on different machines 

- We used pandas to concatenate the files in data.zip folder to a combined file which is 5.6GB in size.
- ** ADD A GENERAL OBSERVATION OF RUN TIMES AMONG THE GROUP MEMBERS**
- ** If anyone runs into issue, specify here...**

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken to combine files |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Arlin   |      MacOS Monterey|  16GB   | 2 GHz Quad-Core Intel Core i5          |    No    |   CPU Time: 5 min 52s; Total: 6 min 9s; Wall time: 6 min 20s
| Artan   |      Windows 10    |  8GB   |   1.80 GHz Intel Core i5                |    Yes    |      CPU times: total: 9min 4s; Wall time: 9min 28s      |
| Shi Yan    |                  |     |           |        |            |


#### 4. Load the combined CSV to memory and perform a simple EDA

EDA question: What is the average predicted percipitation for the `MPI-ESM-1-2-HAM` model?

##### 4.1 Regular Loading

In [2]:
%%time
# Load time
model_name = "MPI-ESM-1-2-HAM"
df = pd.read_csv("figsharerainfall/combined_data.csv")

CPU times: total: 1min 46s
Wall time: 2min 16s


In [7]:
%%time
# Simple EDA
counts = df["model"].value_counts()

CPU times: total: 7.91 s
Wall time: 8.24 s


##### 4.2 Changing Data Type

In [8]:
%%time
# Load time
df = pd.read_csv("figsharerainfall/combined_data.csv", dtype={"lat_min": np.float16,
                                                              "lat_max": np.float16,
                                                              "lon_min": np.float16,  
                                                              "lon_max": np.float16,
                                                              "rain (mm/day)": np.float16})

CPU times: total: 2min 48s
Wall time: 3min 5s


In [9]:
%%time
# Simple EDA
counts = df["model"].value_counts()

CPU times: total: 8.41 s
Wall time: 8.61 s


##### 4.3 Load Selected Columns Only

In [10]:
%%time
# Load time
df = pd.read_csv("figsharerainfall/combined_data.csv", usecols=["model"])

CPU times: total: 1min 34s
Wall time: 1min 36s


In [11]:
%%time
# Simple EDA
counts = df["model"].value_counts()

CPU times: total: 7.5 s
Wall time: 8.18 s


##### 4.4 Loading in Chunks

In [12]:
%%time
# Load and EDA time
chunk_counts = pd.Series(dtype=np.float16)
for chunk in pd.read_csv("figsharerainfall/combined_data.csv", chunksize=10_000_000):
    chunk_counts.add(chunk["model"].value_counts(), 
                     fill_value=None)  # To exclude NaN
chunk_counts.head()

CPU times: total: 2min 40s
Wall time: 2min 50s


Series([], dtype: float16)

##### 4.5 Loading with Dask

In [13]:
%%time
# Load time
import dask.dataframe as dd
df_dask = dd.read_csv("figsharerainfall/combined_data.csv")

CPU times: total: 1.61 s
Wall time: 4.46 s


In [14]:
%%time
#computing values of lat_min variable 
df_dask["model"].value_counts().compute()

CPU times: total: 2min 38s
Wall time: 55 s


MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

##### 4.6 Comparing Wall Time of Different Machines

| Team Member | Task  |  Regular Loading |      Changed Data Type |      Selected Columns |  Chunks            |   Dask        |
|:-----------:|:------|:----------------:|:----------------------:|:---------------------:|:------------------:|:-------------:|
| Arlin       | Load  |                  |                        |                       |                    |               | 
|             | EDA   |                  |                        |                       |                    |               |
| Artan       | Load  |    2min 11s      |        1min 50s        |       47 s            |load + EDA: 2min 50s|    4.46 s     |
|             | EDA   |       8.81s      |        8.61s           |       8.18s           |included above      |    55 s       |
| Shi Yan     | Load  |                  |                        |                       |                    |               |
|             | EDA   |                  |                        |                       |                    |               |

>Observations: Artan's laptop did not have problem loading the csv files but was getting errors like the below when doing an EDA including calculating mean values per model:
>```Python
>MemoryError: Unable to allocate 477. MiB for an array with shape (1, 62467843) and data type object
>```

<br>

#### 5. Perform a simple EDA in R